In [1]:
import pandas as pd
import numpy as np
import pprint
import math
import os
pp = pprint.PrettyPrinter(indent=4).pprint

ex=pd.DataFrame(columns=['ticker','min_lhs' ,'min_rhs', 'max', 'mean', 'mean_last_80', '3/16', '10/40', 'c/min', 'c/max', 'max/min_lhs' ,'max/min_rhs'])

In [2]:
def calc_max(df):
    i_max=df['<HIGH>'].idxmax()
    df=df.reset_index()
    m=df['<HIGH>'].idxmax()
    return {
        "i_max":i_max,
        "max_v":df.iloc[m]['<HIGH>']
    }

In [3]:
def calc_min(df):
    i_min=df['<LOW>'].idxmin()
    df=df.reset_index()
    m=df['<LOW>'].idxmin()
    return {
        "i_min":i_min,
        "min_v":df.iloc[m]['<LOW>']
    }

In [4]:
def calc(data_dir,scv_name):
    SLICE = 360
    I_33=math.floor(.33*SLICE)
    I_50=math.floor(.50*SLICE)

    global ex
    df = pd.read_csv(f"{data_dir}/{scv_name}")
    df = df.replace(0, np.nan)
    df = df.dropna(how='any', axis=0)
    df = df.tail(SLICE)
    df = df.reset_index()
    
    ROWS=df.shape[0]
    if ROWS < SLICE :
        return

    i_min=df['<LOW>'].idxmin()
    i_max=calc_max(df[I_33:-1])['i_max']
    
    min_rhs=calc_min(df[i_max:-1])

    lhs_min_s = 0 if I_33 <= i_max <= I_33 else I_33
    min_lhs=calc_min(df[lhs_min_s:i_max])

    max=df.iloc[i_max]
    min=df.iloc[i_min]

    data={ 
      'ticker':min['<TICKER>'],
      'min_lhs':min_lhs['min_v'],
      'min_rhs':min_rhs['min_v'],
      'max':max['<HIGH>'],        
      'mean':df.tail(240)['<OPENINT>'].mean(),
      'mean_last_80':df.tail(80)['<OPENINT>'].mean(),
      '3/16':df.tail(3)['<OPENINT>'].mean()/df.tail(16).head(13)['<OPENINT>'].mean(),
      '10/40':df.tail(10)['<OPENINT>'].mean()/df.tail(40).head(30)['<OPENINT>'].mean(),
		'c/min':df.at[df.shape[0]-1, '<CLOSE>']/min_rhs['min_v'],
        'c/max':df.at[df.shape[0]-1, '<CLOSE>']/max['<HIGH>'],
		'max/min_rhs':max['<HIGH>']/min_rhs['min_v'],
		'max/min_lhs':max['<HIGH>']/min_lhs['min_v']
    }
    ex =pd.concat([ex,pd.DataFrame(data=[data],columns=data.keys())])
   

In [5]:
for _,_,files in os.walk('data'):
    for f in files:
        if ".csv" in f:
            calc('data/',f)
    ex.to_csv("out.csv")